In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import cv2
import glob
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset
import torch
import torchvision
from torchvision.models import resnet50
import torch.nn as nn
import numpy as np
from torchvision import datasets, models, transforms
from sklearn.svm import SVC

In [2]:
device = "cpu"

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path1 = "/content/gdrive/MyDrive/Train/diseased"

In [6]:
path2 = "/content/gdrive/MyDrive/Train/healthy"

In [7]:
import os
train = []
test = []

c = 0

for filename in os.listdir(path1):
  filename = path1 + "/" + filename
  if c > 564:
    l = 1
    test.append([filename,l])
    c += 1
  else:
    l = 1
    train.append([filename,l])
    c += 1

In [8]:
import os
l = [0,0]

c = 0

for filename in os.listdir(path2):
  filename = path2 + "/" + filename
  if c > 654:
    l = 0
    test.append([filename,l])
    c += 1
  else:
    l = 0
    train.append([filename,l])
    c += 1

In [9]:
class Dataset(Dataset):
    

    def __init__(self, train, root_dir, transform=None):
        
        self.landmarks_frame = train
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        
        img_name = os.path.join(self.root_dir,self.landmarks_frame[idx][0])
        image = cv2.imread(img_name)
        y = torch.tensor(self.landmarks_frame[idx][1])
        y = y.float()
        
        

        if self.transform:
            image = self.transform(image)

        return (image,y)

In [10]:
print(len(train))

1220


In [11]:
print(len(test))

519


In [12]:
train_dataset = Dataset(train = train,root_dir='', transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224, transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

]))

In [13]:
test_dataset = Dataset(train = test,root_dir='', transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224, transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

]))

In [14]:
num_classes = 2
batch_size = 8


In [15]:
trainloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

testloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

In [16]:
model = models.resnet50(pretrained=True)
model.fc = nn.Flatten()

model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
x = []
y = []

In [18]:
for inputs, labels in trainloader:
  with torch.no_grad():
    inputs = inputs.to(device)
    labels = labels.to(device)
    out = model(inputs)
  for z in out:
    x.append(z.numpy())
  labels = labels.numpy()
  for k in labels:    
    y.append(k)


In [19]:
clf = SVC(kernel = "linear")

In [20]:
print(x[0])

[0.5284404  0.15367606 0.7397176  ... 0.15748228 0.55525076 0.47681737]


In [21]:
print(y[0])

0.0


In [22]:
clf.fit(x, y)

SVC(kernel='linear')

In [23]:
X = []
Y = []

In [24]:
for inputs, labels in testloader:
  with torch.no_grad():
    out = model(inputs)
  for z in out:
    X.append(z.numpy())
  labels = labels.numpy()
  for k in labels:
    Y.append(k)

In [25]:
preds = clf.predict(X)
h = 0

for k in range(len(preds)):
  if preds[k] == Y[k]:
    h += 1

print(h/len(preds))

0.9633911368015414


In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
cm = confusion_matrix(Y, preds)

In [29]:
print(cm)

[[270   9]
 [ 10 230]]


In [30]:
from sklearn.metrics import classification_report
y_true = Y
y_pred = preds
target_names = ['class' + str(x) for x in range(2)]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      class0       0.96      0.97      0.97       279
      class1       0.96      0.96      0.96       240

    accuracy                           0.96       519
   macro avg       0.96      0.96      0.96       519
weighted avg       0.96      0.96      0.96       519

